In [1]:
import requests
res = requests.get('https://my83.com.tw/question')

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, 'lxml')

In [53]:
question_list = []
domain = 'https://my83.com.tw'
for section in soup.select('#questionList section'):
    #print(section)
    link = domain + section.select('a')[0]['href']
    question_label, question = section.select('h3')[0].contents
    tags = ','.join([tag.text for tag in section.select('.tag')])
    dt, replies = section.select('.text-right .list-group-item-text')
    userinfo    = section.select('.userInfo')[0].text.strip()
    content     = section.select('.content')[0].text.strip()
    question_list.append({'question_label':question_label.text, \
                          'question'      :question, \
                          'content'       :content, \
                          'dt'            :dt.text, \
                          'replies'       :replies.text, \
                          'tags'          :tags, \
                          'userinfo'      :userinfo,\
                          'link'          :link})


In [54]:
import pandas
questiondf = pandas.DataFrame(question_list)

In [56]:
questiondf.head()

,content,dt,link,question,question_label,replies,tags,userinfo
0,諜對蝶【LINE】：spyvsfly\r\n 看的到素質.教育服務態度【以客為尊】\r\n ...,2017-11-30 04:21:55,https://my83.com.tw/question/12153,台南外約♥諜對蝶LINE：spyvsfly 台南叫小姐,投保問題,0,"其他,保險觀念",201545562
1,諜對蝶【LINE】：spyvsfly\r\n 看的到素質.教育服務態度【以客為尊】\r\n ...,2017-11-30 03:16:49,https://my83.com.tw/question/12152,宜蘭魚訊♥諜對蝶LINE：spyvsfly宜蘭正妹,投保問題,0,"其他,理賠",201545562
2,諜對蝶【LINE】：spyvsfly\r\n 看的到素質.教育服務態度【以客為尊】\r\n ...,2017-11-30 02:56:21,https://my83.com.tw/question/12151,彰化外送茶♥諜對蝶LINE：spyvsfly彰化外約,閒聊,0,"其他,理賠",201545562
3,個人資訊：\r\n年齡 27 / 女性 / 單身\r\n台北市外商文書工作 / 通勤以捷運為...,2017-11-29 21:46:05,https://my83.com.tw/question/12149,27 歲女第一張保單醫療險請益,保單好不好,5,"小資族,實支實付醫療,醫療險,癌症險,重大疾病險",Birdy
4,這些是以前當學生時買的保單(新光人壽)請問可以幫我檢查一下我還需加點甚麼嗎?\r\n目前本人...,2017-11-29 21:42:16,https://my83.com.tw/question/12148,保單健檢,保單好不好,7,小資族,momo77


## 抓取內容

In [57]:
import requests
res = requests.get('https://my83.com.tw/question/12013')

In [59]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, 'html.parser')

In [64]:
question_content = soup.select('.question-content')[0].text
question_content

'33歲 男性 有1子女 請問現有保單有需要增加或減少的地方嗎?\n\r\n富邦人壽真心關懷重大疾病暨特定傷病終身健康保險主約  30萬\r\n富邦人壽防癌終身健康保險附約(PCC1) 3單位\r\n富邦人壽享安心住院醫療定額健康保險附約 1000元\r\n富邦人壽長青健康保險附約(計畫c) 1單位\r\n富邦人壽新住院醫療定期健康保險附約(HSC5) 1單位\r\n富邦人壽安心寶意外傷害保險附約 100萬 \r\n安心寶意外傷害保險附約意外傷害醫療保險金一般型 5萬 \r\n富邦人壽日額型意外傷害住院醫療保險附約 20單位 \r\n富邦人壽保險費豁免附約條款 '

In [81]:
import pandas
answer_ary = []
for answer in soup.select('#answerList .item'):
    author   = answer.select('.consultation')[0].text
    content  = answer.select('.answer-content')[0].text
    identity = answer.select('.col-sm-8 .list-group-item-text')[1].text
    date     = answer.select('.date')[0].text
    answer_ary.append({'author' : author,\
                      'content' : content,\
                      'identity': identity,\
                      'date'    : date})
    #print('============================================')

In [83]:
answerdf = pandas.DataFrame(answer_ary)
answerdf

,author,content,date,identity
0,保險經紀人Roger,富邦人壽真心關懷重大疾病暨特定傷病終身健康保險主約 30萬\n理賠包含癌症在內多項特殊疾病\...,2017-11-22 01:41:09,保險業務員 @ 新北市
1,保險經紀人Ann,VC610您好\n富邦人壽真心關懷重大疾病暨特定傷病終身健康保險主約 30萬\n重大傷病的部...,2017-11-22 02:12:19,保險業務員 @ 高雄市
2,林珉毅,PCC1跟享安心住院可以有更好的選擇\n\n有任何問題或需要服務的地方歡迎來信!\n感謝您!,2017-11-29 20:57:46,保險業務員 @ 台中市 磐石保經


## 整理內容頁

In [84]:
link = 'https://my83.com.tw/question/12013'

In [129]:
import requests
import pandas
import re
import json
import pandas
from bs4 import BeautifulSoup

link = 'https://my83.com.tw/question/12013'
res = requests.get(link)
soup = BeautifulSoup(res.text, 'html.parser')

m = re.search('question: (.*)',res.text)

jd = json.loads(m.group(1).rstrip(','))

output = {}
output['content']      = jd['content']
output['careate_time'] = jd['created_time']
output['nickname']     = jd['nickname']
output['tag_name']     = ','.join([tag['tag_name'] for tag in jd['tag']])
output['title']        = jd['title']
output['category']     = jd['category']
output['link']         = link
output

{'careate_time': '2017-11-22 01:01:51',
 'category': 'policy',
 'content': '33歲 男性 有1子女 請問現有保單有需要增加或減少的地方嗎?<br />\r\n<br />\r\n富邦人壽真心關懷重大疾病暨特定傷病終身健康保險主約  30萬<br />\r\n富邦人壽防癌終身健康保險附約(PCC1) 3單位<br />\r\n富邦人壽享安心住院醫療定額健康保險附約 1000元<br />\r\n富邦人壽長青健康保險附約(計畫c) 1單位<br />\r\n富邦人壽新住院醫療定期健康保險附約(HSC5) 1單位<br />\r\n富邦人壽安心寶意外傷害保險附約 100萬 <br />\r\n安心寶意外傷害保險附約意外傷害醫療保險金一般型 5萬 <br />\r\n富邦人壽日額型意外傷害住院醫療保險附約 20單位 <br />\r\n富邦人壽保險費豁免附約條款 ',
 'link': 'https://my83.com.tw/question/12013',
 'nickname': 'VC610',
 'tag_name': '小家庭,實支實付醫療,住院日額型醫療,意外險,癌症險',
 'title': '33歲 男性 有1子女 請問富邦人壽保單有需要增加或減少的地方嗎?'}

In [130]:
import pandas, sqlite3
df = pandas.DataFrame([output])
#df

with sqlite3.connect('my83.sqlite') as db:
    df.to_sql('question', con = db, if_exists='replace')

In [131]:
with sqlite3.connect('my83.sqlite') as db:
    df2 = pandas.read_sql_query('select * from question', con = db)
df2

,index,careate_time,category,content,link,nickname,tag_name,title
0,0,2017-11-22 01:01:51,policy,33歲 男性 有1子女 請問現有保單有需要增加或減少的地方嗎?<br />\r\n<br /...,https://my83.com.tw/question/12013,VC610,"小家庭,實支實付醫療,住院日額型醫療,意外險,癌症險",33歲 男性 有1子女 請問富邦人壽保單有需要增加或減少的地方嗎?


## 抓取清單頁，將問題存進資料庫中

In [133]:
import requests
import pandas
import re
import json
import pandas
from bs4 import BeautifulSoup

def getQuestions(link):
    res = requests.get(link)
    soup = BeautifulSoup(res.text, 'html.parser')

    m = re.search('question: (.*)',res.text)

    jd = json.loads(m.group(1).rstrip(','))

    output = {}
    output['content']      = jd['content']
    output['careate_time'] = jd['created_time']
    output['nickname']     = jd['nickname']
    output['tag_name']     = ','.join([tag['tag_name'] for tag in jd['tag']])
    output['title']        = jd['title']
    output['category']     = jd['category']
    output['link']         = link
    return output

In [137]:
import requests
from bs4 import BeautifulSoup
question_url = 'https://my83.com.tw/question/index?page={}'
domain = 'https://my83.com.tw'

question_list = []
for page in range(1,4):
    print(page)
    res = requests.get(question_url.format(page))
    soup = BeautifulSoup(res.text, 'lxml')

    for section in soup.select('#questionList section'):
        link = domain + section.select('a')[0]['href']
        question_list.append(getQuestions(link))


1
2
3


In [138]:
import pandas, sqlite3
df = pandas.DataFrame(question_list)
with sqlite3.connect('my83.sqlite') as db:
    df.to_sql('questions', con = db, if_exists='replace') # replace, append

In [140]:
with sqlite3.connect('my83.sqlite') as db:
    df2 = pandas.read_sql_query('select * from questions where content like "%富邦%" ', con = db)
df2

,index,careate_time,category,content,link,nickname,tag_name,title
0,8,2017-11-29 14:18:43,question,富邦nhr1有包含住院前後門診費用嗎 謝謝,https://my83.com.tw/question/12138,,"小家庭,其他,實支實付醫療,理賠",富邦nhr1有包含住院前後門診費用嗎 謝謝
1,23,2017-11-28 11:23:46,question,目前規劃雙實支-其中一家是用全球<br />\r\n全球：LDC(C型) -20年/1萬+X...,https://my83.com.tw/question/12108,fy,"小資族,保險類型,實支實付醫療,醫療險,意外險,癌症險,重大疾病險,定期",MY83精選搭配-富邦.國泰.南山


## 整理回答

In [93]:
question_content = soup.select('.question-content')[0].text
soup.select('.userInfo')[0].text.strip()
soup.select('.label')

[<span class="label label-success">保單好不好</span>,
 <a class="label label-info text-white" href="/company/profile/fubonlife">富邦人壽</a>]

In [ ]:

answer_ary = []
for answer in soup.select('#answerList .item'):
    author   = answer.select('.consultation')[0].text
    content  = answer.select('.answer-content')[0].text
    identity = answer.select('.col-sm-8 .list-group-item-text')[1].text
    date     = answer.select('.date')[0].text
    answer_ary.append({'author' : author,\
                      'content' : content,\
                      'identity': identity,\
                      'date'    : date})
    answerdf = pandas.DataFrame(answer_ary)
answerdf